In [1]:
from sly import Lexer, Parser
import sys

In [2]:
#Tablas donde ira todo:
tablaEnsam = {}
tablaTipos = {}
#Bandera identificacion:
banderaDIGI = False
banderaID = False
banderaOR = False
banderaAND = False
variableGlobal = False
#Bandera para inicializacion:
numVariables = 0
numPrintf = 0
#Etiqueta final:
final = 0
start = 0
notOr = 0
false = 0
#Ver si x operador es ID o NUMERO:
Numa = False #Si True, emptya es un id y no un numero.
Numb = False #Si True, emptyb es un id y no un numero.

In [3]:
class NodoIntro():
    def P():
        for x in tablaEnsam:
            i=tablaEnsam[x]
        i=int(i)
        i=i*-1
        print('.text')
        print('.globl main')
        print('.type main, @function')
        print('main:')
        print('')
        print('push %ebp')
        print('movl %esp, %ebp')
        print('subl ${}, %esp'.format(i))
        print('')

class NodoPrint():
    def P(a):
        global numPrintf
        numPrintf=numPrintf+1
        if a in tablaEnsam:
            print('pushl {}(%ebp)'.format(tablaEnsam[a]))
        else:
            print("Syntax error: ID '%c' isnt in table." %a)
            sys.exit(0)
    def P2():
        global numPrintf
        numVariables2=numPrintf+1
        p = '"'
        while numPrintf>0:
            p=p+'%d'
            numPrintf=numPrintf-1
        p=p+'"'
        print('puhsl {}'.format(p))
        print('call printf')
        print('')
        print('addl ${}, %esp'.format(numVariables2*4))       

class NodoDeclaracion():
    def P(a):
        if variableGlobal == False:
            global numVariables
            numVariables = numVariables+1
            tablaEnsam[a] = -numVariables*4
        else:
            tablaGlobal[a]='None'
        nodo = tablaTipos[a];
        arrayVariables = 1;
        while (isinstance(nodo, NodoArray)):
            arrayVariables = arrayVariables * nodo.cantidad()
            nodo = nodo.contenido()
        
        numVariables = numVariables + arrayVariables - 1

class NodoID():
    def P(a,b):
        if a in tablaEnsam:
            print('movl %eax, {}(%ebp)'.format(tablaEnsam[a]))
            print('')
        else:
            print('Error, ID {} isnt in table.'.format(a))
            sys.exit(0)
        return tablaEnsam[a]

class NodoGen():
    def P(a):
        if a in tablaEnsam:
            print('movl {}(%ebp), %eax'.format(tablaEnsam[a]))
        else:
            print('movl ${}, %eax'.format(a))
        print('')
        return a  

class NodoCmpl():
    def P(a,b):
        global Numa, Numb
        if Numa == True and Numb == True:
            print('movl {}(%ebp), %eax'.format(tablaEnsam[a]))
            print('cmpl {}(%ebp), %eax'.format(tablaEnsam[b]))
        if Numa == False and Numb == True:
            print('movl ${}, %eax'.format(a))
            print('cmpl {}(%ebp), %eax'.format(tablaEnsam[b]))
        if Numa == True and Numb == False:
            print('movl {}(%ebp), %eax'.format(tablaEnsam[a]))
            print('cmpl ${}, %eax'.format(b))          
        if Numa == False and Numb == False:
            print('movl ${}, %eax'.format(a))
            print('cmpl ${}, %eax'.format(b))           

class NodoWhile():
    def P():
        global start
        print('start{}:'.format(start))
    def P2():
        global final, start, notOr, banderaOR
        if banderaOR == True:
            print('jmp start{}:'.format(start))
            print('notOr{}:'.format(notOr))
            notOr=notOr+1
        else:    
            print('jmp start{}:'.format(start))
            print('final{}:'.format(final))
            final=final+1
        print('')
        start=start+1

class NodoMenor():
    def P(a,b):
        global final, notOr, banderaOR
        if banderaOR == True:
            print('jl final{}'.format(final))
            print('jmp notOr{}'.format(notOr))
        else:
            print('jge final{}'.format(final))
        print('')

class NodoMayor():
    def P(a,b):
        global final, notOr, banderaOR
        if banderaOR == True:
            print('jg final{}'.format(final))
            print('jmp notOr{}'.format(notOr))
        else:
            print('jle final{}'.format(final))
        print('')
    
class NodoMenorIg():
    def P(a,b):
        global final, notOr, banderaOR
        if banderaOR == True:
            print('jle final{}'.format(final))
            print('jmp notOr{}'.format(notOr))
        else:
            print('jg final{}'.format(final))
        print('')
        
class NodoMayorIg():
    def P(a,b):
        global final, notOr, banderaOR
        if banderaOR == True:
            print('jge final{}'.format(final))
            print('jmp notOr{}'.format(notOr))
        else:
            print('jl final{}'.format(final))
        print('')
        
class NodoIg():
    def P(a,b):
        global final, notOr, banderaOR
        if banderaOR == True:
            print('je final{}'.format(final))
            print('jmp notOr{}'.format(notOr))
        else:
            print('jne final{}'.format(final))
        print('')
        
class NodoNot():
    def P(a):
        global final
        print('compl $0, %eax')
        print('je final{}'.format(final))
        print('movl $1, %eax')
        print('')
    
class NodoSum():
    def P(a,b):
        global Numa, Numb
        print('movl %eax, %ebx')
        print('popl %eax')
        print('addl %ebx, %eax')
        print('')
        return a
        
class NodoResta():
    def P(a,b):
        print('movl %eax, %ebx')
        print('popl %eax')
        print('subl %ebx, %eax')
        print('')
        return a
    
class NodoMul():
    def P(a,b):
        print('movl %eax, %ebx')
        print('popl %eax')
        print('imull %ebx, %eax')
        print('')
        return a
    
class NodoDiv():
    def P(a,b):
        global Numa, Numb
        print('mobvl %eax, %ebx')
        print('popl %eax')
        print('cdq')
        print('idivl %ebx')
        print('')
        return a

class NodoPush():
    def P():
        print('pushl %eax')
        print('')

class NodoRU():
    def P(a):
        global Numa
        if Numa == True:
            print('movl {}(%ebp), %eax'.format(tablaEnsam[a]))
            print('imull -1, %eax')
        else:
            print('movl %{}, %eax'.format(a))
            print('imull -1, %eax')            
        return a
    
class NodoTabla():
    def P(a):
        return a
    
class NodoReturn():
    def P():
        print('movl $0, %eax')
        print('')
        print('movl %ebp, %esp')
        print('popl %ebp')
        print('ret')

class NodoIf():
    def P():
        global final, notOr, banderaOR
        if banderaOR == True:
            print('notOr{}:'.format(notOr))
            notOr=notOr+1
        else:
            print('final{}:'.format(final)) #final1:
            final=final+1
        print('')
    def P2():
        global final
        final=final+1
        print('jmp final{}'.format(final)) #jmp final1
        final=final-1
        print('final{}:'.format(final)) #final0:
        final=final+1      
        
class NodoInt():
    def Escribir(self):
        print('Entero.')
    def etiqueta():
        return 'int'
    
class NodoPuntero():
    Nodo = None
    def __init__(self, Nodo):
        self.Nodo = Nodo
    def Escribir(self):
        print("Puntero a ")
        self.Nodo.Escribir()
    def etiqueta(self):
        return self.Nodo.etiqueta()
    
class NodoArray():
    Nodo = None
    cant = 0
    def __init__(self, Nodo, cte):
        self.Nodo = Nodo
        self.cant = cte
    def Escribir(self):
        print("NodoArray de", self.cant, " ")
        self.Nodo.Escribir()
    def etiqueta(self):
        return self.Nodo.etiqueta()
    def cantidad(self):
        return self.cant
    def contenido(self):
        return self.Nodo
    
class NodoDefinicionTipo():
    def P(ID, Nodo):
        tablaTipos[ID] = Nodo
        #print('Var ', ID, ':')
        #Nodo.Escribir()
        

In [4]:
class CalcLexer(Lexer):
    tokens = {DIGIT, ZERO, ID, LESEQ, BIGEQ, EQ, PRINTF, PGENERIC, IF , ELSE, WHILE, MAIN, RETURN, INT}
    ignore = ' \t\n'
    ZERO = r'0'
    DIGIT = r'[0-9]+'
    ELSE = r'else'
    PRINTF = r'printf'
    PGENERIC = r'%d'
    INT = r'int'
    MAIN = r'main'
    IF = r'if'
    WHILE = r'while'
    RETURN = r'return'
    ID = r'[a-zA-Z][a-zA-Z0-9]*'
    
    LESEQ = r'<='
    BIGEQ = r'>='
    EQ = r'=='
    
    literals = {'+', '-', '*', '/', '(', ')', '<', '>', '=', ';', '|', '&', '!', ',', '"', '{', '}','[',']'}
    
    @_(r'0')
    def ZERO(self, f):
        f.value=int(f.value)
        return f
    
    @_(r'[0-9]+')
    def DIGIT(self, f):
        f.value=int(f.value)
        return f
    
    def ID(self, f):
        f.value=str(f.value)
        return f
    
    def error(self, t):
        print('Bad character %r' % t.value[0])
        sys.exit(0)    

In [1]:
class CalcParser(Parser):
    tokens=CalcLexer.tokens
    def __init__(self):
        self.names={}
    
    @_('INT MAIN "(" ")" "{" definiciones intro entrada returnS  "}"')
    def entradaPrincipal(self, p):
        pass
    
    @_('definicion ";" definiciones')
    def definiciones(self,p):
        pass
    
    @_('')
    def definiciones(self,p):
        pass
    
    @_('tipo lista')
    def definicion(self, p):
        pass
    
    @_('INT')
    def tipo(self, p):
        return NodoInt()
    
    @_('elm resto')
    def lista(self, p):
        return p[-1]
    
    @_('"," emptyElm elm resto')
    def resto(self, p):
        pass
        
    @_('')
    def emptyElm(self,p):
        return p[-3]
    
    @_('')
    def resto(self, p):
        pass
    
    @_('"*" emptyPointer elm')
    def elm(self, p):
        pass
    
    @_('')
    def emptyPointer(self, p):
        return NodoPuntero(p[-2])
        
        
    @_('ID emptyPreArray array')
    def elm(self, p):
        NodoDefinicionTipo.P(p.ID, p.array)
        NodoDeclaracion.P(p.ID)
        pass
    
    @_('')
    def emptyPreArray(self,p):
        return p[-2]
    
    @_('"[" DIGIT "]" emptyArray array')
    def array(self, p):
        return NodoArray(p.array, p.DIGIT)
    
    @_('')
    def emptyArray(self,p):
        return p[-4]
    
    @_('')
    def array(self, p):
        return p[-1];
    
    @_('entrada datos')
    def entrada(self, p):
        pass
    
    @_('')
    def intro(self, p):
        NodoIntro.P()
        
    @_('entrada sentenciaIF')
    def entrada(self, p):
        pass
    
    @_('entrada sentenciaWHILE')
    def entrada(self, p):
        pass
    
    @_('')
    def entrada(self, p):
        pass
    
    @_('WHILE emptyWHILE "(" expr ")" "{" entrada "}"')
    def sentenciaWHILE(self, p):
        NodoWhile.P2()
        pass
    
    @_('')
    def emptyWHILE(self, p):
        NodoWhile.P()
        pass
    
    @_('IF "(" expr ")" "{" entrada "}"')
    def sentenciaIF(self, p):
        NodoIf.P()
        pass
        
    @_('IF "(" expr ")" "{" entrada "}" emptyIfElse elseS')
    def sentenciaIF(self, p):
        NodoIf.P()
        pass
    
    @_('ELSE  "{" entrada "}" ')
    def elseS(self, p):
        pass
    
    @_('')
    def emptyIfElse(self, p):

        NodoIf.P2()
        pass
    
    @_('PRINTF "(" """ PGENERIC mas ID ")" ";" ')
    def datos(self, p):
        NodoPrint.P(p.ID)
        NodoPrint.P2()
        pass
    
    @_('var "=" comp emptya ";"')
    def datos(self, p):

        NodoID.P(p.var, p.comp)
        pass
    
    @_('""" ","')
    def mas(self,p):
        pass
    
    @_('PGENERIC mas ID ","')
    def mas(self ,p):
        NodoPrint.P(p.ID)    
    
    @_('expr emptyOr "|" "|" expr')
    def expr(self, p):
        global final
        print('final{}:'.format(final))
        final=final+1
        pass      
    
    @_('')
    def emptyOr(self, p):
        global banderaOR
        banderaOR = True
        pass
    
    @_('expr1')
    def expr(self, p):
        pass
    
    @_('expr1 emptyAnd "&" "&" expr1')
    def expr1(self, p):
        pass
    
    @_('')
    def emptyAnd(self, p):
        global banderaAND
        banderaAND = True
        pass
        
    @_('expr2')
    def expr1(self, p):
        pass
    
    @_('expr2 emptya "<" comp emptyb')
    def expr2(self, p):
        NodoCmpl.P(p.expr2, p.comp)
        NodoMenor.P(p.expr2, p.comp)
        pass
    
    @_('expr2 emptya ">" comp emptyb')
    def expr(self, p):
        NodoCmpl.P(p.expr2, p.comp)
        NodoMayor.P(p.expr2, p.comp)
        pass
    
    @_('expr2 emptya LESEQ comp emptyb')
    def expr2(self, p):
        NodoCmpl.P(p.expr2, p.comp)
        NodoMenorIg.P(p.expr2, p.comp)
        pass
    
    @_('expr2 emptya BIGEQ comp emptyb')
    def expr2(self, p):
        NodoCmpl.P(p.expr2, p.comp)
        NodoMayorIg.P(p.expr2, p.comp)
        pass
    
    @_('expr2 emptya EQ comp emptyb')
    def expr2(self, p):
        NodoCmpl.P(p.expr2, p.comp)
        NodoIg.P(p.expr2, p.comp)
        pass        
  
    @_('comp')
    def expr2(self, p):
        return p.comp
    
    @_('comp emptyPush "+" sum')
    def comp(self, p):
        return NodoSum.P(p.comp, p.sum)
            
    @_('comp emptyPush "-" sum')
    def comp(self, p):

        return NodoResta.P(p.comp, p.sum)
    
    @_('')
    def emptyPush(self, p):
        NodoPush.P()
        
    @_('sum')
    def comp(self, p):
        return p.sum
    
    @_('sum emptyPush "*" fact')
    def sum(self, p):
        return NodoMul.P(p.sum, p.fact)
    
    @_('sum emptyPush "/" fact')
    def sum(self, p):
        return NodoDiv.P(p.sum, p.fact)
        
    @_('fact')
    def sum(self, p):
        return p.fact
    
    @_('"-" fact emptyPush')
    def fact(self, p):
        return NodoRU.P(p.fact)
    
    @_('')
    def emptya(self, p):
        global Numa
        if p[-1] in tablaEnsam:
            Numa = True
        else:
            Numa = False
            
    @_('')
    def emptyb(self, p):
        global Numb
        if p[-1] in tablaEnsam:
            Numb = True
        else:
            Numb = False
    
    @_('var')
    def fact(self, p):
        return NodoTabla.P(p.var)

    @_('DIGIT')
    def fact(self, p):
        return NodoGen.P(p.DIGIT)
    
    @_('"(" expr ")"')
    def fact(self, p):
        pass
    
    @_('"*" var')
    def var(self,p):
        return p.var
    
    @_('ID varArray')
    def var(self,p):
        return p.ID
    
    @_('"[" DIGIT "]" varArray')
    def varArray(self,p):
        pass
    
    @_('')
    def varArray(self,p):
        pass

    @_('RETURN ZERO ";"')
    def returnS(self, p):
        NodoReturn.P()
        pass

NameError: name 'Parser' is not defined

In [6]:
if __name__=='__main__':
    lexer=CalcLexer() 
    parser=CalcParser()
    data=open('Prueba8.c')
    parser.parse(lexer.tokenize(data.read()))

Var  a :
Entero.
Var  b :
Puntero a 
Entero.
Var  c :
Puntero a 
Puntero a 
Puntero a 
Entero.
Var  d :
NodoArray de 3  
Puntero a 
Entero.
Var  e :
NodoArray de 4  
NodoArray de 2  
Entero.
.text
.globl main
.type main, @function
main:

push %ebp
movl %esp, %ebp
subl $28, %esp

movl $0, %eax

movl %ebp, %esp
popl %ebp
ret


In [7]:
tablaTipos

{'a': <__main__.NodoInt at 0x29ca16f5908>,
 'b': <__main__.NodoPuntero at 0x29ca16f5278>,
 'c': <__main__.NodoPuntero at 0x29ca16f54a8>,
 'd': <__main__.NodoArray at 0x29ca16f58d0>,
 'e': <__main__.NodoArray at 0x29ca16f5860>}